In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import numpy as np


In [ ]:
df_kfoldavg =  pd.read_csv("C:\\Users\\Andreas Langholz\\Yieldcaster\\ \
    data\\outputs\\model_performance\\model_perf1.csv", index_col=0)

In [ ]:
fig = go.Figure([go.Scatter(x=df_kfoldavg['year'], y=df_kfoldavg['train_rmse'],
                            marker_color='green',
                            opacity=0.6,
                            name='Value 1')])

fig.add_traces([go.Scatter(x=df_kfoldavg['year'], y=df_kfoldavg['test_rmse'],
                           marker_color='blue',
                           name='Value 2')])

fig.show()